In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import math
import pprint as pp

In [ ]:
wit_sizes = [84, 384, 512, 1024, 2048]

# rerun ped with 20 and sha with 10 iters (instead of 10, 5)
samples = [20, 20, 10, 10, 10]
sonic = {
    "Prove": [0.371, 1.610, 47.736, 76.899, 126.918],
    "Verify": [0.00123, 0.00649, 0.457, 0.727, 1.272],
    "Batch Verify Total": [0.0173, 0.0225, 0.469, 0.742, 1.280], # to recalculate helped verify where negative
    "Helped Verify": [0.000844, 0.000843, 0.00130, 0.00167, 0.000822]
}
lamassu = {
    "Prove": [0.661, 1.938, 49.400, 79.517, 126.597],
    "Verify": [0.00282, 0.00804, 0.455, 0.734, 1.277],
    "Batch Verify Total": [0.0183, 0.0236, 0.454, 0.730, 1.277],
    "Helped Verify": [0.000816, 0.000821, 
                      0, 0, # NOTE: negative so code couldn't calculate
                      0.0000132], 
}

bb_lamassu = {
    "Prove": [5.758, 6.592, 56.729, 89.510, 155.982],
    "Verify": [0.0226, 0.0226, 0.490, 0.770, 1.349],
    "Batch Verify Total": [0.0363, 0.0381, 0.494, 0.783, 1.355],
    "Helped Verify": [0.000719, 0.000817, 0.000347, 0.00142, 0.000666],
}

In [ ]:
for benchmarks in [sonic, lamassu, bb_lamassu]:
    for i in range(len(benchmarks["Helped Verify"])):
        if benchmarks["Helped Verify"][i] == 0: # i.e. code errored out because negative
            benchmarks["Helped Verify"][i] = (benchmarks["Batch Verify Total"][i] - benchmarks["Verify"][i])/(samples[i]-1)
    pp.pprint(benchmarks)

In [ ]:
ucse_overhead = {
    key: [bb_lamassu[key][i]/sonic[key][i] for i in range(len(bb_lamassu[key]))] for key in sonic.keys()
}
ucse_overhead

In [ ]:
fig = plt.figure()
row1 = fig.add_subplot(211)
# Turn off axis lines and ticks of row1
row1.spines['top'].set_color('none')
row1.spines['left'].set_color('none')
row1.spines['right'].set_color('none')
row1.spines['bottom'].set_color('none')
row1.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)

row2 = fig.add_subplot(212)
# Turn off axis lines and ticks of row2
row2.spines['top'].set_color('none')
row2.spines['left'].set_color('none')
row2.spines['right'].set_color('none')
row2.spines['bottom'].set_color('none')
row2.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)

plot_order = ["Prove", "Verify"]#, "Batch Verify Total"]#, "Helped Verify"]

### row 1 (times)
for i in range(len(plot_order)):
    ax1 = fig.add_subplot(2,3,i+1)
    ax1.set_xticks(ticks=[])
    # pedersen
    ax1.plot(wit_sizes[:2], sonic[plot_order[i]][:2], 'o-', label="Sonic")
    ax1.plot(wit_sizes[:2], lamassu[plot_order[i]][:2], 'o-', label="Lamassu")
    ax1.plot(wit_sizes[:2], bb_lamassu[plot_order[i]][:2], 'o-', label="This work")

    ax2 = ax1.twinx()
    # sha256
    ax2.plot(wit_sizes[2:len(sonic[plot_order[i]])], sonic[plot_order[i%3]][2:], 'o--', label="Sonic")
    ax2.plot(wit_sizes[2:len(lamassu[plot_order[i]])], lamassu[plot_order[i%3]][2:], 'o--', label="Lamassu")
    ax2.plot(wit_sizes[2:len(bb_lamassu[plot_order[i]])], bb_lamassu[plot_order[i%3]][2:], 'o--', label="This work")
    
    ax1.set_title(plot_order[i])
    
handles, labels = ax1.get_legend_handles_labels()
handles += [Line2D([0], [0], color='tab:grey', ls='-', label='Pedersen'),
            Line2D([0], [0], color='tab:grey', ls='--', label='SHA-256')]
labels += ['Pedersen', 'SHA-256']

ax1.legend(handles, labels, loc="lower right")

# left y-label
row1.set_ylabel("Pedersen time (s)")
# right y-label
ax2.set_ylabel("SHA-256 time (s)")

### row 2 (overhead)
for i in range(len(plot_order)):
    ax = fig.add_subplot(2,3,i+4)
    ax.set_xticks(ticks=wit_sizes)
    ax.plot(wit_sizes[:len(ucse_overhead[plot_order[i]])], ucse_overhead[plot_order[i]], 'o-', color='k')
    ax.set_xticklabels(wit_sizes, rotation=50, ha="right")

row2.set_xlabel("Witness size (bits)")
row2.set_ylabel("UC SE overhead")
fig.set_figheight(7)
width = 12 if len(plot_order) == 2 else 15
fig.set_figwidth(width)
plt.tight_layout()

# plt.show()
plt.savefig("benchmarks.png")

In [ ]:
### Prover time plot
fig, ax1 = plt.subplots()
plt.xticks(ticks=wit_sizes)
# pedersen
ax1.plot(wit_sizes[:2], sonic["Prove"][:2], 'o-', label="Sonic")
ax1.plot(wit_sizes[:2], lamassu["Prove"][:2], 'o-', label="Lamassu")
ax1.plot(wit_sizes[:2], bb_lamassu["Prove"][:2], 'o-', label="This work")
# ax1.legend()

ax2 = ax1.twinx()
ax2.set_ylim([700, 2000])
# sha256
ax2.plot(wit_sizes[2:], sonic["Prove"][2:], 'o--', label="Sonic")
ax2.plot(wit_sizes[2:], lamassu["Prove"][2:], 'o--', label="Lamassu")
ax2.plot(wit_sizes[2:], bb_lamassu["Prove"][2:], 'o--', label="This work")
# ax2.legend()

# legend
handles, labels = ax1.get_legend_handles_labels()
handles += [Line2D([0], [0], color='tab:grey', ls='-', label='Pedersen'),
                Line2D([0], [0], color='tab:grey', ls='--', label='SHA-256')]
labels += ['Pedersen', 'SHA-256']
    
plt.legend(handles, labels)
plt.xlabel("witness size (bits)")
ax1.set_ylabel("Pedersen Time (s)")
ax2.set_ylabel("SHA-256 Time (s)")
plt.title("Prove")

plt.show()
# plt.savefig("prover_time.png")

In [ ]:
### Prover time overhead
plt.xticks(ticks=wit_sizes)
plt.plot(wit_sizes, ucse_overhead["Prove"], 'o-', color='k')
plt.title("Prove")
plt.xlabel("witness size (bits)")
plt.ylabel("UC SE overhead")

plt.show()
# plt.savefig("ucse_overhead_prover.png")